Outline:

1. Introduce the data gathering and parsing
1. How to create a VIT from the GPT code
    1. Change to an encoder
    1. Handling the position embedding
    1. Adding the text goals
    1. Binning the cotinuous values
1. Sanitising the data and standardization.
1. Adding goal images
1. Adding the input masking
1. Evaluating the model in sim
1. Recording videos of the results for evaluation

## Get full trajectories
Instead of a single image we want something that looks more like a sequence, similar to text. 
For robotics applications our "language" is images and actions.

# Load Datasets

The data for robotics applications is often more complicated. There are images, actions and text descriptions. Also, the text descriptions is per episode, instead of at each frame, which is common for RL/BC.

Show some of the patching with images.

## Postition Encodings


## Transformer Code

In [ ]:
from hydra import compose, initialize
initialize(config_path="./conf", job_name="test_app")

In [5]:

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

import tensorflow_datasets as tfds
import numpy as np
from tqdm import tqdm, trange
import cv2


# data loading
def get_batch_vit(split, dataset, batch_size):
    # generate a small batch of inputs x and targets y
    data = dataset['train'] if split == 'train' else dataset['test']
    ix = np.random.randint(int(len(data["img"])), size=(batch_size,))
    x = torch.tensor(data["img"][ix], dtype=torch.float)
    y = torch.tensor(data["label"][ix], dtype=torch.long)
    # x, y = x.to(device), y.to(device)
    return x, y

def calc_positional_embeddings(sequence_length, d):
    result = torch.ones(sequence_length, d)
    for i in range(sequence_length):
        for j in range(d):
            result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
    return result

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(model._cfg.eval_iters)
        for k in range(model._cfg.eval_iters):
            X, Y = get_batch_vit(split, model._dataset, model._cfg.batch_size)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

def get_patches_fast(images):
    from einops import rearrange
    batch_size, channels, height, width = images.shape
    patch_size = height // 8 ## n_patches = 8

    p = patch_size # P in maths

    patches = rearrange(images, 'b (h p1) (w p2) c -> b (h w) (p1 p2 c)', p1 = p, p2 = p)
    return patches

def calc_positional_embeddings(sequence_length, d):
    result = torch.ones(sequence_length, d)
    for i in range(sequence_length):
        for j in range(d):
            result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
    return result

## This is an encoder head (full attention)
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size, n_embd, dropout):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        # wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) ## Remove masking
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size, n_embd, dropout):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embd=n_embd, dropout=dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head, dropout):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, n_embd=n_embd, dropout=dropout)
        self.ffwd = FeedFoward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class VIT(nn.Module):
  def __init__(self, dataset, cfg, mlp_ratio=4):
    super(VIT, self).__init__()
    self._dataset = dataset
    self._cfg = cfg
    # assert shape[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"
    # assert shape[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"
    self.patch_size = (cfg.image_shape[0] / cfg.n_patches, cfg.image_shape[1] / cfg.n_patches)

    #Positional embedding
    # self.pos_embed = nn.Parameter(torch.tensor(positional_embeddings(n_patches ** 2 + 1, embedding_size)))
    # self. pos_embed.requires_grad = False
    self.register_buffer('positional_embeddings', calc_positional_embeddings(cfg.n_patches ** 2 + 1, cfg.n_embd), persistent=False)
    # self.position_embedding_table = nn.Embedding(n_patches ** 2 + 1, n_embd)
    
    self.class_tokens = nn.Parameter(torch.rand(1, cfg.n_embd))

    self.input_d = int(cfg.image_shape[2] * self.patch_size[0] * self.patch_size[1])

    self.lin_map = nn.Linear(self.input_d, cfg.n_embd, bias=False) 

    # 4) Transformer encoder blocks
    self.blocks = nn.ModuleList([Block(cfg.n_embd, cfg.n_head, dropout=self._cfg.dropout) for _ in range(cfg.n_blocks)])

    # 5) Classification MLPk
    self.mlp = nn.Sequential(
        nn.Linear(cfg.n_embd, cfg.action_bins),
        nn.Softmax(dim=-1)
    )

  def forward(self, images, targets=None):
    # Dividing images into patches
    n, c, h, w = images.shape
    patches = get_patches_fast(images)
    
    # Running linear layer tokenization
    # Map the vector corresponding to each patch to the hidden size dimension
    out = self.lin_map(patches)
    
    # Adding classification token to the tokens
    out = torch.cat((self.class_tokens.expand(n, 1, -1), out), dim=1)
    
    # Adding positional embedding
    out = out + self.positional_embeddings.repeat(n, 1, 1)
    # pos_emb = self.position_embedding_table(torch.arange(n_patches ** 2 + 1, device=device)) # (T,C)
    # out = out + pos_emb
    
    # Transformer Blocks
    for block in self.blocks:
        out = block(out)

    # Getting the classification token only
    out = out[:, 0]
    logits = self.mlp(out)
        
    if targets is None:
        loss = None
    else:
        # B,T,C = 4,8,2 # batch, time, channels
        B, C = logits.shape
        # logits = logits.view(B*T, C)
        targets = targets.view(B)
        loss = F.cross_entropy(logits, targets)
    return (logits, loss)

# import hydra, json

# @hydra.main(config_path="conf", config_name="vit-64")
def my_main():
    
    from hydra import compose, initialize
    from omegaconf import DictConfig, OmegaConf
    # initialize(config_path="./conf", job_name="test_app")
    cfg = compose(config_name="vit-64", overrides=["+env=absolute_path"])

    torch.manual_seed(cfg.r_seed)
    print ("cfg:", OmegaConf.to_yaml(cfg))
    # print (vars(cfg))
    print (OmegaConf.to_container(cfg))
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")

    from datasets import load_dataset
    ds = load_dataset(cfg.dataset)

    print('Features:', ds["train"].features)
    # np.reshape(np.array(x["img"][i].getdata(), dtype=np.float32)
    trim = 1000000 ## Lets see how little data is needed to still get good performance. 1000 is not enough.
    dataset = {}
    dataset["train"]= {
            "img": torch.tensor(np.array(ds["train"]["img"][:trim], dtype=np.uint8)).to(device),
            "label": torch.tensor(np.array(ds["train"]["label"][:trim], dtype=np.uint8)).to(device) 
            }         
    dataset["test"]=  {
            "img": torch.tensor(np.array(ds["test"]["img"][:trim], dtype=np.uint8)).to(device),
            "label": torch.tensor(np.array(ds["test"]["label"][:trim], dtype=np.uint8)).to(device)
            }
    # print ("Results:", results)
    import wandb
    # start a new wandb run to track this script
    wandb.init(
        # set the wandb project where this run will be logged
        project="mini-vit",

        # track hyperparameters and run metadata
        config= OmegaConf.to_container(cfg)
    )
    wandb.run.log_code(".")
    model = VIT(dataset, cfg)
    m = model.to(device)
    # print the number of parameters in the model
    print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

    # create a PyTorch optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)

    for iter in range(cfg.max_iters):

        # every once in a while evaluate the loss on train and val sets
        if iter % cfg.eval_interval == 0 or iter == cfg.max_iters - 1:
            losses = estimate_loss(model)
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
            wandb.log({"train loss": losses['train'], "val loss": losses['val']})

        # sample a batch of data
        xb, yb = get_batch_vit('train', dataset, cfg.batch_size)

        # evaluate the loss
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # generate from the model
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    # print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))
    wandb.finish()
    return losses['val']

if __name__ == "__main__":
    import os
    results = my_main()
    print("results:", results)

cfg: batch_size: 64
block_size: 32
n_patches: 8
max_iters: 10000
eval_interval: 100
learning_rate: 0.0001
eval_iters: 200
n_embd: 256
r_seed: 1337
n_head: 16
n_blocks: 4
dropout: 0.1
action_bins: 10
image_shape:
- 64
- 64
- 3
dataset: EleutherAI/cifarnet
trim: 1000000
error: false
env: absolute_path

{'batch_size': 64, 'block_size': 32, 'n_patches': 8, 'max_iters': 10000, 'eval_interval': 100, 'learning_rate': 0.0001, 'eval_iters': 200, 'n_embd': 256, 'r_seed': 1337, 'n_head': 16, 'n_blocks': 4, 'dropout': 0.1, 'action_bins': 10, 'image_shape': [64, 64, 3], 'dataset': 'EleutherAI/cifarnet', 'trim': 1000000, 'error': False, 'env': 'absolute_path'}
Using device:  cuda (Quadro RTX 8000)
Features: {'img': Image(mode=None, decode=True, id=None), 'label': ClassLabel(names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], id=None)}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gberseth (real-lab). Use `wandb login --relogin` to force relogin


3.207946 M parameters


/tmp/ipykernel_2366172/3511894213.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(data["img"][ix], dtype=torch.float)
/tmp/ipykernel_2366172/3511894213.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(data["label"][ix], dtype=torch.long)


step 0: train loss 2.3035, val loss 2.3038
step 100: train loss 2.2440, val loss 2.2439
step 200: train loss 2.2139, val loss 2.2181
step 300: train loss 2.2002, val loss 2.2109
step 400: train loss 2.1853, val loss 2.1817
step 500: train loss 2.1662, val loss 2.1742
step 600: train loss 2.1565, val loss 2.1596
step 700: train loss 2.1505, val loss 2.1620
step 800: train loss 2.1545, val loss 2.1593
step 900: train loss 2.1261, val loss 2.1337
step 1000: train loss 2.1257, val loss 2.1271
step 1100: train loss 2.0997, val loss 2.1052
step 1200: train loss 2.1139, val loss 2.1135
step 1300: train loss 2.1004, val loss 2.1131
step 1400: train loss 2.0794, val loss 2.0967
step 1500: train loss 2.0942, val loss 2.1096
step 1600: train loss 2.0579, val loss 2.0581
step 1700: train loss 2.0766, val loss 2.0887
step 1800: train loss 2.0951, val loss 2.0958
step 1900: train loss 2.0732, val loss 2.0819
